In [8]:
from collections import Counter

a = "arvand"
b = "arvana"

counter_a = Counter(a)
counter_b = Counter(b)

In [9]:
print(set(counter_a))

{'r', 'v', 'n', 'd', 'a'}


In [10]:
all = set(counter_a) | set(counter_b)
all

{'a', 'd', 'n', 'r', 'v'}

In [11]:
raw_diff = sum(abs(counter_a[l] - counter_b[l]) for l in all)
raw_diff

2

In [12]:
n_diff = abs(len(a) - len(b))
n_diff

0

In [13]:
    # substitutions = the paired mismatches beyond size difference
num_subs = (raw_diff - n_diff) // 2
num_subs

1

In [14]:
    # total node cost = substitutions + unmatched ins/dels
delta_v = num_subs + n_diff
delta_v

1